# Benchmark

Whenever we make changes to the code we will rerun a simple benchmark in order to keep track of how the simulation results might change over time. It is in general hard to verify that the equations are implemented correctly because there exist no analytic solutions to the equations. However, by running the same benchmark over time we can see if the changes we make will impact the simulation results. Here the different biomarkers are taken from traces evaluated a the center of at slab that is simulated for 1000 ms (see the benchmark in the library for more details)

In [2]:
from pathlib import Path
import json
import matplotlib.pyplot as plt
import numpy as np
import datetime
from collections import defaultdict
import plotly
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.options.display.max_columns = 5

In [2]:
benchmark_folder = Path("__file__").absolute().parent.parent / "benchmarks"

In [3]:
all_hashes = [f.name for f in benchmark_folder.iterdir()]

In [4]:
print(all_hashes)

['c7df02b', '0c195ff', '4d3a653', 'd5edd92', 'f63afc9', '630ed09', 'cf7cc9d']


You can checkout a specific commit by going to the link

https://github.com/ComputationalPhysiology/simcardems/commit/GIT_COMMIT_HASH

where you swap out GIT_COMMIT_HASH with the specific hash, i.e

In [111]:
for hash in all_hashes:
    print(f"https://github.com/ComputationalPhysiology/simcardems/commit/{hash}")

https://github.com/ComputationalPhysiology/simcardems/commit/c7df02b
https://github.com/ComputationalPhysiology/simcardems/commit/0c195ff
https://github.com/ComputationalPhysiology/simcardems/commit/4d3a653
https://github.com/ComputationalPhysiology/simcardems/commit/d5edd92
https://github.com/ComputationalPhysiology/simcardems/commit/f63afc9
https://github.com/ComputationalPhysiology/simcardems/commit/630ed09
https://github.com/ComputationalPhysiology/simcardems/commit/cf7cc9d


In [5]:
data = {}
for git_hash in all_hashes:
    results_file = benchmark_folder / git_hash / "results.json"
    if not results_file.is_file():
        print(f"Results does not exist for version {git_hash}")
        continue
    with open(results_file, "r") as f:
        data[git_hash] = json.load(f)
        

In [21]:
results = defaultdict(list)

for git_hash, d in data.items():
    results["git_hash"].append(git_hash)
    for key, value in d.items():
        results[key].append(value)

In [104]:
df = pd.DataFrame(results)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df

,git_hash,import_time,timestamp,simcardems_version,create_runner_time,solve_time,maxTa,ampTa,APD40,APD50,...,max_displacement_y,rel_max_displacement_y,max_displacement_perc_y,rel_max_displacement_perc_y,time_to_max_displacement_y,max_displacement_z,rel_max_displacement_z,max_displacement_perc_z,rel_max_displacement_perc_z,time_to_max_displacement_z
0,c7df02b,9.000000e-07,2022-10-06 20:28:25.015389,2022.3.0,194.206491,727.766001,1.154621,1.154621,238.847917,265.374952,...,0.0,0.049886,0.0,0.249432,138.05,0.0,0.049886,0.0,0.249432,138.05
1,0c195ff,1.000000e-06,2022-10-06 20:19:12.227464,2022.3.0,211.102322,755.622428,1.154621,1.154621,238.847917,265.374952,...,0.0,0.049886,0.0,0.249432,138.05,0.0,0.049886,0.0,0.249432,138.05
2,4d3a653,1.100000e-06,2022-10-06 20:09:07.230901,2022.3.0,205.289198,784.220401,1.154621,1.154621,238.847917,265.374952,...,0.0,0.049886,0.0,0.249432,138.05,0.0,0.049886,0.0,0.249432,138.05
3,d5edd92,3.330060e-07,2022-10-07 06:42:09.101000,2022.3.0,10.719169,3198.345828,1.148763,1.148763,238.850807,265.377778,...,0.0,0.049752,0.0,0.248758,138.05,0.0,0.049752,0.0,0.248758,138.05
4,f63afc9,1.001000e-06,2022-10-06 16:25:06.830650,2022.3.0,237.720608,854.584107,1.154621,1.154621,238.847917,265.374952,...,0.0,0.049886,0.0,0.249432,138.05,0.0,0.049886,0.0,0.249432,138.05
5,630ed09,1.000000e-06,2022-10-06 16:55:04.711879,2022.3.0,280.511200,1019.038654,1.154621,1.154621,238.847917,265.374952,...,0.0,0.049886,0.0,0.249432,138.05,0.0,0.049886,0.0,0.249432,138.05
6,cf7cc9d,9.000000e-07,2022-10-06 20:32:00.361561,2022.3.0,206.295999,745.851108,1.154621,1.154621,238.847917,265.374952,...,0.0,0.049886,0.0,0.249432,138.05,0.0,0.049886,0.0,0.249432,138.05


In [108]:
df = df.sort_values(by='timestamp')
df

,git_hash,import_time,timestamp,simcardems_version,create_runner_time,solve_time,maxTa,ampTa,APD40,APD50,...,max_displacement_y,rel_max_displacement_y,max_displacement_perc_y,rel_max_displacement_perc_y,time_to_max_displacement_y,max_displacement_z,rel_max_displacement_z,max_displacement_perc_z,rel_max_displacement_perc_z,time_to_max_displacement_z
4,f63afc9,1.001000e-06,2022-10-06 16:25:06.830650,2022.3.0,237.720608,854.584107,1.154621,1.154621,238.847917,265.374952,...,0.0,0.049886,0.0,0.249432,138.05,0.0,0.049886,0.0,0.249432,138.05
5,630ed09,1.000000e-06,2022-10-06 16:55:04.711879,2022.3.0,280.511200,1019.038654,1.154621,1.154621,238.847917,265.374952,...,0.0,0.049886,0.0,0.249432,138.05,0.0,0.049886,0.0,0.249432,138.05
2,4d3a653,1.100000e-06,2022-10-06 20:09:07.230901,2022.3.0,205.289198,784.220401,1.154621,1.154621,238.847917,265.374952,...,0.0,0.049886,0.0,0.249432,138.05,0.0,0.049886,0.0,0.249432,138.05
1,0c195ff,1.000000e-06,2022-10-06 20:19:12.227464,2022.3.0,211.102322,755.622428,1.154621,1.154621,238.847917,265.374952,...,0.0,0.049886,0.0,0.249432,138.05,0.0,0.049886,0.0,0.249432,138.05
0,c7df02b,9.000000e-07,2022-10-06 20:28:25.015389,2022.3.0,194.206491,727.766001,1.154621,1.154621,238.847917,265.374952,...,0.0,0.049886,0.0,0.249432,138.05,0.0,0.049886,0.0,0.249432,138.05
6,cf7cc9d,9.000000e-07,2022-10-06 20:32:00.361561,2022.3.0,206.295999,745.851108,1.154621,1.154621,238.847917,265.374952,...,0.0,0.049886,0.0,0.249432,138.05,0.0,0.049886,0.0,0.249432,138.05
3,d5edd92,3.330060e-07,2022-10-07 06:42:09.101000,2022.3.0,10.719169,3198.345828,1.148763,1.148763,238.850807,265.377778,...,0.0,0.049752,0.0,0.248758,138.05,0.0,0.049752,0.0,0.248758,138.05


In [38]:
df[["simcardems_version", "timestamp", "git_hash"]]

,simcardems_version,timestamp,git_hash
4,2022.3.0,2022-10-06T16:25:06.830650,f63afc9
5,2022.3.0,2022-10-06T16:55:04.711879,630ed09
2,2022.3.0,2022-10-06T20:09:07.230901,4d3a653
1,2022.3.0,2022-10-06T20:19:12.227464,0c195ff
0,2022.3.0,2022-10-06T20:28:25.015389,c7df02b
6,2022.3.0,2022-10-06T20:32:00.361561,cf7cc9d
3,2022.3.0,2022-10-07T06:42:09.101000,d5edd92


In [112]:
columns = [c for c in df.columns if c not in ["timestamp", "simcardems_version", "git_hash"]]
git_hash = df["git_hash"]
versions = df["simcardems_version"]
dates = [t.date() for t in df["timestamp"]]

text = []
for h,v, t in zip(git_hash, versions, dates):
    text.append("\n".join([
        f"<br>Git Hash: {h}</br>",
        f"<br>Version: {v}</br>",
        f"<br>Timestamp {t}</br>",
    ]))
    
fig = make_subplots(rows=12, cols=4, subplot_titles=columns)

for i, c in enumerate(columns):
    
    row = i % 12 + 1
    col = i // 12 + 1
    
    fig.add_trace(
        go.Scatter(x=df["timestamp"], y=df[c], text=text, hovertemplate="%{text}"),
        row=row, col=col
    )
    
fig.update_layout(
    width=1000,
    height=3500,
    showlegend=False
)
fig.show()